In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import seaborn as sns
import time
import h5py
import pandas as pd
import seaborn as sns
from pandas import DataFrame

import tensorflow as tf
from tensorflow.keras import layers, models
import random

# Set a specific random seed
random.seed(42)

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
torch.cuda.get_device_name(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = torch.tensor([1.0, 2.0, 3.0])
x = x.to(device)
print(x.device)
torch.cuda.get_device_name(0)

cuda:0


'NVIDIA GeForce RTX 2060'

In [4]:
import torch
import numpy as np

# Set seed for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


# loading the DS002 dataset

In [5]:
# Time tracking, Operation time (min):  0.003
filename="N-CMAPSS_DS02-006.h5"
t = time.process_time()  

# Load data
with h5py.File(filename, 'r') as hdf:
        # Development set
        W_dev = np.array(hdf.get('W_dev'))             # W
        X_s_dev = np.array(hdf.get('X_s_dev'))         # X_s
        X_v_dev = np.array(hdf.get('X_v_dev'))         # X_v
        T_dev = np.array(hdf.get('T_dev'))             # T
        Y_dev = np.array(hdf.get('Y_dev'))             # RUL  
        A_dev = np.array(hdf.get('A_dev'))             # Auxiliary

        # Test set
        W_test = np.array(hdf.get('W_test'))           # W
        X_s_test = np.array(hdf.get('X_s_test'))       # X_s
        X_v_test = np.array(hdf.get('X_v_test'))       # X_v
        T_test = np.array(hdf.get('T_test'))           # T
        Y_test = np.array(hdf.get('Y_test'))           # RUL  
        A_test = np.array(hdf.get('A_test'))           # Auxiliary
        
        # Varnams
        W_var = np.array(hdf.get('W_var'))
        X_s_var = np.array(hdf.get('X_s_var'))  
        X_v_var = np.array(hdf.get('X_v_var')) 
        T_var = np.array(hdf.get('T_var'))
        A_var = np.array(hdf.get('A_var'))
        
        # from np.array to list dtype U4/U5
        W_var = list(np.array(W_var, dtype='U20'))
        X_s_var = list(np.array(X_s_var, dtype='U20'))  
        X_v_var = list(np.array(X_v_var, dtype='U20')) 
        T_var = list(np.array(T_var, dtype='U20'))
        A_var = list(np.array(A_var, dtype='U20'))
                          
W = np.concatenate((W_dev, W_test), axis=0)  
X_s = np.concatenate((X_s_dev, X_s_test), axis=0)
X_v = np.concatenate((X_v_dev, X_v_test), axis=0)
T = np.concatenate((T_dev, T_test), axis=0)
Y = np.concatenate((Y_dev, Y_test), axis=0) 
A = np.concatenate((A_dev, A_test), axis=0) 
    
print('')
print("Operation time (min): " , (time.process_time()-t)/60)
print('')
print ("W shape: " + str(W.shape))
print ("X_s shape: " + str(X_s.shape))
print ("X_v shape: " + str(X_v.shape))
print ("T shape: " + str(T.shape))
print ("A shape: " + str(A.shape))


Operation time (min):  0.30052083333333335

W shape: (6517190, 4)
X_s shape: (6517190, 14)
X_v shape: (6517190, 14)
T shape: (6517190, 10)
A shape: (6517190, 4)


In [6]:
df_W = DataFrame(data=W, columns=W_var)
df_X_s_parameters = DataFrame(data=X_s, columns=X_s_var)
df_X_v_parameters = DataFrame(data=X_v, columns=X_v_var)
df_T = DataFrame(data=T, columns=T_var)
df_Y = DataFrame(data=Y, columns=['RUL'])
df_A = DataFrame(data=A, columns=A_var)
df_unit1=pd.concat([df_A,df_X_s_parameters,df_X_v_parameters,df_T,df_Y],axis=1)
def prepare_data(dataframe,list_of_columns_to_drop):
    dataframe=dataframe.drop(columns=list_of_columns_to_drop)
    return dataframe
def data_test_train_splitter(DataFrame,train_idx,test_idx,list_of_columns_to_drop):
    train_df=prepare_data(DataFrame.loc[DataFrame["unit"].isin(train_idx)],list_of_columns_to_drop)
    test_df=prepare_data(DataFrame.loc[DataFrame["unit"].isin(test_idx)],list_of_columns_to_drop)
    return (train_df,test_df)

In [27]:
12383/60

206.38333333333333

In [26]:
df_unit1.loc[(df_unit1["unit"]==5) & (df_unit1["cycle"]==5) ]

,unit,cycle,Fc,hs,T24,T30,T48,T50,P15,P2,...,fan_flow_mod,LPC_eff_mod,LPC_flow_mod,HPC_eff_mod,HPC_flow_mod,HPT_eff_mod,HPT_flow_mod,LPT_eff_mod,LPT_flow_mod,RUL
895356,5.0,5.0,3.0,1.0,605.415527,1449.491663,1833.549522,1234.221535,16.231743,11.859551,...,0.0,0.0,0.0,0.0,0.0,-0.000755,0.0,0.0,0.0,84
895357,5.0,5.0,3.0,1.0,605.390722,1449.436713,1833.351521,1234.055132,16.228119,11.856384,...,0.0,0.0,0.0,0.0,0.0,-0.000755,0.0,0.0,0.0,84
895358,5.0,5.0,3.0,1.0,605.375621,1449.396643,1833.357153,1234.066198,16.225811,11.854989,...,0.0,0.0,0.0,0.0,0.0,-0.000755,0.0,0.0,0.0,84
895359,5.0,5.0,3.0,1.0,605.368840,1449.372692,1833.307833,1234.018671,16.224054,11.853761,...,0.0,0.0,0.0,0.0,0.0,-0.000755,0.0,0.0,0.0,84
895360,5.0,5.0,3.0,1.0,605.434935,1449.462974,1833.493910,1234.119184,16.229294,11.858560,...,0.0,0.0,0.0,0.0,0.0,-0.000755,0.0,0.0,0.0,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907734,5.0,5.0,3.0,1.0,549.717298,1212.366823,1395.886472,1050.282698,13.302806,11.616100,...,0.0,0.0,0.0,0.0,0.0,-0.000755,0.0,0.0,0.0,84
907735,5.0,5.0,3.0,1.0,549.747191,1212.411133,1395.766625,1050.196906,13.306581,11.619138,...,0.0,0.0,0.0,0.0,0.0,-0.000755,0.0,0.0,0.0,84
907736,5.0,5.0,3.0,1.0,549.721489,1212.413239,1395.790805,1050.257089,13.305935,11.617879,...,0.0,0.0,0.0,0.0,0.0,-0.000755,0.0,0.0,0.0,84
907737,5.0,5.0,3.0,1.0,549.801230,1212.557861,1396.142076,1050.504337,13.314397,11.626024,...,0.0,0.0,0.0,0.0,0.0,-0.000755,0.0,0.0,0.0,84


In [7]:
train_idx=[2]
test_idx=[14,11,15]
list_of_columns_to_drop=["cycle","hs",'fan_eff_mod','fan_flow_mod','LPC_eff_mod','HPC_eff_mod','HPC_flow_mod','HPT_flow_mod','LPT_eff_mod','LPT_flow_mod','LPC_flow_mod']
train,test=data_test_train_splitter(df_unit1,train_idx,test_idx,list_of_columns_to_drop)
train.drop(columns=["unit","Fc"])
test.drop(columns=["unit","Fc"])
train


,unit,Fc,T24,T30,T48,T50,P15,P2,P21,P24,...,W31,W32,W48,W50,SmFan,SmLPC,SmHPC,phi,HPT_eff_mod,RUL
0,2.0,3.0,600.148034,1438.498187,1818.027714,1228.129848,15.806267,11.577097,16.046971,20.126624,...,26.498785,15.899271,215.844851,228.411666,16.648833,9.898130,25.376144,41.893990,-0.000638,74
1,2.0,3.0,600.055894,1438.350208,1817.682618,1227.879113,15.795477,11.568235,16.036017,20.113218,...,26.486552,15.891931,215.745634,228.307014,16.639222,9.904927,25.380549,41.884434,-0.000638,74
2,2.0,3.0,600.210756,1439.109101,1820.020627,1229.422522,15.807747,11.574866,16.048474,20.130956,...,26.519340,15.911604,216.019054,228.592279,16.649823,9.923503,25.318848,41.953848,-0.000638,74
3,2.0,3.0,600.369717,1439.240230,1819.188327,1228.538726,15.816360,11.578198,16.057218,20.146716,...,26.532044,15.919226,216.121238,228.702994,16.653812,9.905518,25.361981,41.914342,-0.000638,74
4,2.0,3.0,600.298227,1439.064004,1818.963540,1228.389046,15.807513,11.571593,16.048236,20.135888,...,26.518460,15.911076,216.008509,228.584788,16.649031,9.897465,25.363994,41.911503,-0.000638,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853137,2.0,3.0,549.873954,1228.114477,1470.594274,1096.724266,13.116898,11.199832,13.316648,15.164883,...,16.738961,10.043377,135.696974,143.719571,16.959057,6.783033,31.180818,34.338442,-0.018145,0
853138,2.0,3.0,549.907672,1228.177220,1470.739956,1096.830687,13.120695,11.203387,13.320503,15.169186,...,16.742474,10.045484,135.725237,143.749289,16.964797,6.782442,31.176976,34.342768,-0.018145,0
853139,2.0,3.0,549.923591,1228.209936,1470.703869,1096.792601,13.122416,11.204765,13.322250,15.171310,...,16.744940,10.046964,135.745765,143.771001,16.966299,6.782323,31.179834,34.339757,-0.018145,0
853140,2.0,3.0,549.947285,1228.250165,1470.788949,1096.864006,13.125276,11.207342,13.325154,15.174606,...,16.747638,10.048583,135.767263,143.793778,16.967980,6.780795,31.178010,34.342228,-0.018145,0


### LSTM with Hidden size=32 and CNN channels=32 Loss on all=114

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import time

# Assuming df_unit1 is already defined
features = train.drop(columns=['RUL',"unit","Fc"]).values
target = train['RUL'].values.reshape(-1, 1)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

class CustomDataset(Dataset):
    def __init__(self, data, targets):
        self.data = torch.tensor(data, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, cnn_channels):
        super(Model, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True,bidirectional=False)
        self.conv1d = nn.Conv1d(in_channels=1, out_channels=cnn_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(hidden_size + cnn_channels, 1)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        cnn_out = self.conv1d(x.unsqueeze(1))
        cnn_out = self.relu(cnn_out)
        cnn_out = torch.max_pool1d(cnn_out, kernel_size=cnn_out.size(2))
        cnn_out = cnn_out.squeeze(2)
        combined_out = torch.cat((lstm_out, cnn_out), dim=1)
        output = self.fc(combined_out)
        return output

# # Explicitly set the device to CPU
# device = torch.device("cpu")
# print(f"Training on device: {device}")

input_size = features_scaled.shape[1] 
hidden_size = 32 
num_layers = 2  
cnn_channels = 32 
batch_size = 64 
learning_rate = 0.001 
num_epochs = 10  


dataset = CustomDataset(features_scaled, target)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
model = Model(input_size, hidden_size, num_layers, cnn_channels).to(device)  # Move model to the device

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

start_time = time.time()

for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)  # Move data to the device
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")
# on the test all print("losss= 114.60372838793877")
# on 14 ,,35.01807977637466

# BiLSTM with hidden=64 and CNN channels=32 Loss=84 you get 76 if you only use unit 2 for training


In [16]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import time

# Assuming df_unit1 is already defined
features = train.drop(columns=['RUL', "unit", "Fc"]).values
target = train['RUL'].values.reshape(-1, 1)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

class CustomDataset(Dataset):
    def __init__(self, data, targets):
        self.data = torch.tensor(data, dtype=torch.float32).unsqueeze(1)  # Add sequence dimension if needed
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, cnn_channels):
        super(Model, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.conv1d = nn.Conv1d(in_channels=1, out_channels=cnn_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(hidden_size * 2 + cnn_channels, 1)

    def forward(self, x):
#         print(f"Input shape: {x.shape}")
        lstm_out, _ = self.lstm(x)
#         print(f"LSTM output shape: {lstm_out.shape}")
        # Use the output from the last time step for LSTM output
        lstm_out = lstm_out[:, -1, :]  # Shape: (batch_size, hidden_size * 2)
#         print(f"LSTM last time step output shape: {lstm_out.shape}")

        # CNN expects input of shape (batch_size, in_channels, seq_len)
        cnn_in = x.squeeze(1).unsqueeze(1)  # Shape: (batch_size, 1, seq_len)
#         print(f"CNN input shape: {cnn_in.shape}")
        cnn_out = self.conv1d(cnn_in)  # Shape: (batch_size, cnn_channels, seq_len)
#         print(f"CNN output shape before pooling: {cnn_out.shape}")
        cnn_out = self.relu(cnn_out)
        cnn_out = torch.max_pool1d(cnn_out, kernel_size=cnn_out.size(2)).squeeze(2)
#         print(f"CNN output shape after pooling: {cnn_out.shape}")

        # Concatenate LSTM and CNN outputs along the feature dimension
        combined_out = torch.cat((lstm_out, cnn_out), dim=1)
#         print(f"Combined output shape: {combined_out.shape}")
        output = self.fc(combined_out)
        return output

# Training parameters
input_size = features_scaled.shape[1]
hidden_size = 64
num_layers = 2
cnn_channels = 32
batch_size = 64
learning_rate = 0.001
num_epochs = 10

# Dataset and DataLoader
dataset = CustomDataset(features_scaled, target)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Model, criterion, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model(input_size, hidden_size, num_layers, cnn_channels).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
start_time = time.time()

for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")
# on all 84.91848756703507
# on 14 33.31
# hidden=64====> on all 76.67486999544822 for train i used only unit 2

Epoch [1/10], Loss: 7.2884
Epoch [2/10], Loss: 4.3562
Epoch [3/10], Loss: 3.2852
Epoch [4/10], Loss: 4.4299
Epoch [5/10], Loss: 1.6721
Epoch [6/10], Loss: 0.1865
Epoch [7/10], Loss: 2.9905
Epoch [8/10], Loss: 0.1893
Epoch [9/10], Loss: 0.6834
Epoch [10/10], Loss: 0.2889
Training time: 509.66 seconds


# BiLSTM with Hidden size=32 and CNN=32 and Attention(XLSTM) Loss=74,97037406072378 i trained only using unit 2

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import time

# Assuming df_unit1 is already defined
features = train.drop(columns=['RUL', "unit", "Fc"]).values
target = train['RUL'].values.reshape(-1, 1)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

class CustomDataset(Dataset):
    def __init__(self, data, targets):
        self.data = torch.tensor(data, dtype=torch.float32).unsqueeze(1)  # Add sequence dimension if needed
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

class XLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, cnn_channels, attn_size):
        super(XLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.layer_norm = nn.LayerNorm(hidden_size * 2)
        self.attention = nn.Linear(hidden_size * 2, attn_size)
        self.attn_combine = nn.Linear(attn_size, 1)
        self.conv1d = nn.Conv1d(in_channels=1, out_channels=cnn_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(hidden_size * 2 + cnn_channels, 1)

    def forward(self, x):
        print(f"Input shape: {x.shape}")
        lstm_out, _ = self.lstm(x)
        print(f"LSTM output shape: {lstm_out.shape}")
        lstm_out = lstm_out[:, -1, :]  # Use the output from the last time step
        lstm_out = self.layer_norm(lstm_out)
        print(f"LSTM last time step output shape: {lstm_out.shape}")

        attn_weights = torch.tanh(self.attention(lstm_out))
        attn_weights = self.attn_combine(attn_weights)
        attn_weights = torch.softmax(attn_weights, dim=1)
        attn_output = torch.bmm(attn_weights.unsqueeze(1), lstm_out.unsqueeze(1))
        attn_output = attn_output.squeeze(1)
        print(f"Attention output shape: {attn_output.shape}")

        cnn_in = x.squeeze(1).unsqueeze(1)
        print(f"CNN input shape: {cnn_in.shape}")
        cnn_out = self.conv1d(cnn_in)
        print(f"CNN output shape before pooling: {cnn_out.shape}")
        cnn_out = self.relu(cnn_out)
        cnn_out = torch.max_pool1d(cnn_out, kernel_size=cnn_out.size(2)).squeeze(2)
        print(f"CNN output shape after pooling: {cnn_out.shape}")

        combined_out = torch.cat((attn_output, cnn_out), dim=1)
        print(f"Combined output shape: {combined_out.shape}")
        output = self.fc(combined_out)
        return output

# Training parameters
input_size = features_scaled.shape[1]
hidden_size = 32
num_layers = 2
cnn_channels = 32
attn_size = 64
batch_size = 64
learning_rate = 0.001
num_epochs = 10

# Dataset and DataLoader
dataset = CustomDataset(features_scaled, target)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Model, criterion, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = XLSTM(input_size, hidden_size, num_layers, cnn_channels, attn_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
start_time = time.time()

for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")
# 74.97037406072378 on all test and was trained only using 2


# BiLSTM Hidden=64 and CNN=32 and attention trained only on unit 2 loss= 75.89781064193672

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import time

# Assuming df_unit1 is already defined
features = train.drop(columns=['RUL', "unit", "Fc"]).values
target = train['RUL'].values.reshape(-1, 1)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

class CustomDataset(Dataset):
    def __init__(self, data, targets):
        self.data = torch.tensor(data, dtype=torch.float32).unsqueeze(1)  # Add sequence dimension if needed
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

class XLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, cnn_channels, attn_size):
        super(XLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.layer_norm = nn.LayerNorm(hidden_size * 2)
        self.attention = nn.Linear(hidden_size * 2, attn_size)
        self.attn_combine = nn.Linear(attn_size, 1)
        self.conv1d = nn.Conv1d(in_channels=1, out_channels=cnn_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(hidden_size * 2 + cnn_channels, 1)

    def forward(self, x):
        print(f"Input shape: {x.shape}")
        lstm_out, _ = self.lstm(x)
        print(f"LSTM output shape: {lstm_out.shape}")
        lstm_out = lstm_out[:, -1, :]  # Use the output from the last time step
        lstm_out = self.layer_norm(lstm_out)
        print(f"LSTM last time step output shape: {lstm_out.shape}")

        attn_weights = torch.tanh(self.attention(lstm_out))
        attn_weights = self.attn_combine(attn_weights)
        attn_weights = torch.softmax(attn_weights, dim=1)
        attn_output = torch.bmm(attn_weights.unsqueeze(1), lstm_out.unsqueeze(1))
        attn_output = attn_output.squeeze(1)
        print(f"Attention output shape: {attn_output.shape}")

        cnn_in = x.squeeze(1).unsqueeze(1)
        print(f"CNN input shape: {cnn_in.shape}")
        cnn_out = self.conv1d(cnn_in)
        print(f"CNN output shape before pooling: {cnn_out.shape}")
        cnn_out = self.relu(cnn_out)
        cnn_out = torch.max_pool1d(cnn_out, kernel_size=cnn_out.size(2)).squeeze(2)
        print(f"CNN output shape after pooling: {cnn_out.shape}")

        combined_out = torch.cat((attn_output, cnn_out), dim=1)
        print(f"Combined output shape: {combined_out.shape}")
        output = self.fc(combined_out)
        return output

# Training parameters
input_size = features_scaled.shape[1]
hidden_size = 64
num_layers = 2
cnn_channels = 32
attn_size = 64
batch_size = 64
learning_rate = 0.001
num_epochs = 10

# Dataset and DataLoader
dataset = CustomDataset(features_scaled, target)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Model, criterion, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = XLSTM(input_size, hidden_size, num_layers, cnn_channels, attn_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
start_time = time.time()

for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")



In [9]:
# Assuming df_unit1 is already defined
def evaluation(test):
    features = test.drop(columns=['RUL',"unit","Fc"]).values
    target = test['RUL'].values.reshape(-1, 1)
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    dataset = CustomDataset(features_scaled, target)
    test_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)








    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    i=0
    with torch.no_grad():  # Disable gradient computation
        for inputs, targets in test_dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            i=i+1
            test_loss += loss.item()

    # Calculate average test loss
    test_loss /= len(test_dataloader)
    return test_loss

In [12]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
rfe = RFE(model, n_features_to_select=10)  # Select the top 10 features
fit = rfe.fit(features_scaled, target)

selected_features = fit.transform(features_scaled)
selected_feature_names = train.columns[fit.support_]
selected_feature_names


c:\Users\lenovo legion\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\lenovo legion\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\lenovo legion\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\lenovo legion\AppData\Local\Programs\Python\Python310\lib\site-packages\s

KeyboardInterrupt: 

In [16]:
fit = rfe.fit(features_scaled, target)
selected_features = fit.transform(features_scaled)
selected_feature_names = train.columns[fit.support_]
selected_feature_names


c:\Users\lenovo legion\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


KeyboardInterrupt: 

In [14]:
selected_feature_names

NameError: name 'selected_feature_names' is not defined

In [11]:
evaluation(test)

111.32087716861648

# BiLSTM hidden=64 CNN=64 attention trained only on unit 2 loss=72.60894099338253

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import time

# Assuming df_unit1 is already defined
features = train.drop(columns=['RUL', "unit", "Fc"]).values
target = train['RUL'].values.reshape(-1, 1)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

class CustomDataset(Dataset):
    def __init__(self, data, targets):
        self.data = torch.tensor(data, dtype=torch.float32).unsqueeze(1)  # Add sequence dimension if needed
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

class XLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, cnn_channels, attn_size):
        super(XLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.layer_norm = nn.LayerNorm(hidden_size * 2)
        self.attention = nn.Linear(hidden_size * 2, attn_size)
        self.attn_combine = nn.Linear(attn_size, 1)
        self.conv1d = nn.Conv1d(in_channels=1, out_channels=cnn_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(hidden_size * 2 + cnn_channels, 1)

    def forward(self, x):
        print(f"Input shape: {x.shape}")
        lstm_out, _ = self.lstm(x)
        print(f"LSTM output shape: {lstm_out.shape}")
        lstm_out = lstm_out[:, -1, :]  # Use the output from the last time step
        lstm_out = self.layer_norm(lstm_out)
        print(f"LSTM last time step output shape: {lstm_out.shape}")

        attn_weights = torch.tanh(self.attention(lstm_out))
        attn_weights = self.attn_combine(attn_weights)
        attn_weights = torch.softmax(attn_weights, dim=1)
        attn_output = torch.bmm(attn_weights.unsqueeze(1), lstm_out.unsqueeze(1))
        attn_output = attn_output.squeeze(1)
        print(f"Attention output shape: {attn_output.shape}")

        cnn_in = x.squeeze(1).unsqueeze(1)
        print(f"CNN input shape: {cnn_in.shape}")
        cnn_out = self.conv1d(cnn_in)
        print(f"CNN output shape before pooling: {cnn_out.shape}")
        cnn_out = self.relu(cnn_out)
        cnn_out = torch.max_pool1d(cnn_out, kernel_size=cnn_out.size(2)).squeeze(2)
        print(f"CNN output shape after pooling: {cnn_out.shape}")

        combined_out = torch.cat((attn_output, cnn_out), dim=1)
        print(f"Combined output shape: {combined_out.shape}")
        output = self.fc(combined_out)
        return output

# Training parameters
input_size = features_scaled.shape[1]
hidden_size = 64
num_layers = 2
cnn_channels = 64
attn_size = 64
batch_size = 64
learning_rate = 0.001
num_epochs = 10

# Dataset and DataLoader
dataset = CustomDataset(features_scaled, target)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Model, criterion, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = XLSTM(input_size, hidden_size, num_layers, cnn_channels, attn_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
start_time = time.time()

for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")



# BiLSTM hidden=64 CNN=64 attention trained the entireity of DS002 loss=195

In [19]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import time

# Assuming df_unit1 is already defined
features = train.drop(columns=['RUL', "unit", "Fc"]).values
target = train['RUL'].values.reshape(-1, 1)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

class CustomDataset(Dataset):
    def __init__(self, data, targets):
        self.data = torch.tensor(data, dtype=torch.float32).unsqueeze(1)  # Add sequence dimension if needed
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

class XLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, cnn_channels, attn_size):
        super(XLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.layer_norm = nn.LayerNorm(hidden_size * 2)
        self.attention = nn.Linear(hidden_size * 2, attn_size)
        self.attn_combine = nn.Linear(attn_size, 1)
        self.conv1d = nn.Conv1d(in_channels=1, out_channels=cnn_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(hidden_size * 2 + cnn_channels, 1)

    def forward(self, x):
#         print(f"Input shape: {x.shape}")
        lstm_out, _ = self.lstm(x)
#         print(f"LSTM output shape: {lstm_out.shape}")
        lstm_out = lstm_out[:, -1, :]  # Use the output from the last time step
        lstm_out = self.layer_norm(lstm_out)
#         print(f"LSTM last time step output shape: {lstm_out.shape}")

        attn_weights = torch.tanh(self.attention(lstm_out))
        attn_weights = self.attn_combine(attn_weights)
        attn_weights = torch.softmax(attn_weights, dim=1)
        attn_output = torch.bmm(attn_weights.unsqueeze(1), lstm_out.unsqueeze(1))
        attn_output = attn_output.squeeze(1)
#         print(f"Attention output shape: {attn_output.shape}")

        cnn_in = x.squeeze(1).unsqueeze(1)
#         print(f"CNN input shape: {cnn_in.shape}")
        cnn_out = self.conv1d(cnn_in)
#         print(f"CNN output shape before pooling: {cnn_out.shape}")
        cnn_out = self.relu(cnn_out)
        cnn_out = torch.max_pool1d(cnn_out, kernel_size=cnn_out.size(2)).squeeze(2)
#         print(f"CNN output shape after pooling: {cnn_out.shape}")

        combined_out = torch.cat((attn_output, cnn_out), dim=1)
#         print(f"Combined output shape: {combined_out.shape}")
        output = self.fc(combined_out)
        return output

# Training parameters
input_size = features_scaled.shape[1]
hidden_size = 64
num_layers = 2
cnn_channels = 64
attn_size = 64
batch_size = 64
learning_rate = 0.001
num_epochs = 10

# Dataset and DataLoader
dataset = CustomDataset(features_scaled, target)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Model, criterion, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = XLSTM(input_size, hidden_size, num_layers, cnn_channels, attn_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
start_time = time.time()

for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")



Epoch [1/10], Loss: 25.6386
Epoch [2/10], Loss: 12.7439
Epoch [3/10], Loss: 40.8834
Epoch [4/10], Loss: 11.0838
Epoch [5/10], Loss: 11.0820
Epoch [6/10], Loss: 4.9783
Epoch [7/10], Loss: 14.3930
Epoch [8/10], Loss: 4.9185
Epoch [9/10], Loss: 10.0067
Epoch [10/10], Loss: 6.6674
Training time: 3119.24 seconds


# BiLSTM hidden=16 CNN=16 with attention trained on unit2 only loss=73.28647924435874

In [23]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import time

# Assuming df_unit1 is already defined
features = train.drop(columns=['RUL', "unit", "Fc"]).values
target = train['RUL'].values.reshape(-1, 1)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

class CustomDataset(Dataset):
    def __init__(self, data, targets):
        self.data = torch.tensor(data, dtype=torch.float32).unsqueeze(1)  # Add sequence dimension if needed
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

class XLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, cnn_channels, attn_size):
        super(XLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.layer_norm = nn.LayerNorm(hidden_size * 2)
        self.attention = nn.Linear(hidden_size * 2, attn_size)
        self.attn_combine = nn.Linear(attn_size, 1)
        self.conv1d = nn.Conv1d(in_channels=1, out_channels=cnn_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(hidden_size * 2 + cnn_channels, 1)

    def forward(self, x):
#         print(f"Input shape: {x.shape}")
        lstm_out, _ = self.lstm(x)
#         print(f"LSTM output shape: {lstm_out.shape}")
        lstm_out = lstm_out[:, -1, :]  # Use the output from the last time step
        lstm_out = self.layer_norm(lstm_out)
#         print(f"LSTM last time step output shape: {lstm_out.shape}")

        attn_weights = torch.tanh(self.attention(lstm_out))
        attn_weights = self.attn_combine(attn_weights)
        attn_weights = torch.softmax(attn_weights, dim=1)
        attn_output = torch.bmm(attn_weights.unsqueeze(1), lstm_out.unsqueeze(1))
        attn_output = attn_output.squeeze(1)
#         print(f"Attention output shape: {attn_output.shape}")

        cnn_in = x.squeeze(1).unsqueeze(1)
#         print(f"CNN input shape: {cnn_in.shape}")
        cnn_out = self.conv1d(cnn_in)
#         print(f"CNN output shape before pooling: {cnn_out.shape}")
        cnn_out = self.relu(cnn_out)
        cnn_out = torch.max_pool1d(cnn_out, kernel_size=cnn_out.size(2)).squeeze(2)
#         print(f"CNN output shape after pooling: {cnn_out.shape}")

        combined_out = torch.cat((attn_output, cnn_out), dim=1)
#         print(f"Combined output shape: {combined_out.shape}")
        output = self.fc(combined_out)
        return output

# Training parameters
input_size = features_scaled.shape[1]
hidden_size = 16
num_layers = 2
cnn_channels = 16
attn_size = 32
batch_size = 64
learning_rate = 0.001
num_epochs = 10

# Dataset and DataLoader
dataset = CustomDataset(features_scaled, target)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Model, criterion, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = XLSTM(input_size, hidden_size, num_layers, cnn_channels, attn_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
start_time = time.time()

for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")



Epoch [1/10], Loss: 7.2250
Epoch [2/10], Loss: 1.5078
Epoch [3/10], Loss: 4.1231
Epoch [4/10], Loss: 5.8868
Epoch [5/10], Loss: 4.4517
Epoch [6/10], Loss: 4.8269
Epoch [7/10], Loss: 3.7699
Epoch [8/10], Loss: 1.9653
Epoch [9/10], Loss: 1.4357
Epoch [10/10], Loss: 0.7280
Training time: 620.33 seconds


# BiLSTM hidden=32 CNN=32 with attention  and a dropout rate=0.3 trained on unit2 only loss=47.05092177008901 and the second time i got 38.9295559518005 third time i got 34.09007194503465

In [35]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import time

# Assuming df_unit1 is already defined
features = train.drop(columns=['RUL', "unit", "Fc"]).values
target = train['RUL'].values.reshape(-1, 1)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

class CustomDataset(Dataset):
    def __init__(self, data, targets):
        self.data = torch.tensor(data, dtype=torch.float32).unsqueeze(1)  # Add sequence dimension if needed
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

class XLSTM_V2(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, cnn_channels, attn_size, dropout_rate=0.5):
        super(XLSTM_V2, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.layer_norm = nn.LayerNorm(hidden_size * 2)
        self.attention = nn.Linear(hidden_size * 2, attn_size)
        self.attn_combine = nn.Linear(attn_size, 1)
        self.conv1d = nn.Conv1d(in_channels=1, out_channels=cnn_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(hidden_size * 2 + cnn_channels, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out[:, -1, :]  # Use the output from the last time step
        lstm_out = self.layer_norm(lstm_out)

        attn_weights = torch.tanh(self.attention(lstm_out))
        attn_weights = self.attn_combine(attn_weights)
        attn_weights = torch.softmax(attn_weights, dim=1)
        attn_output = torch.bmm(attn_weights.unsqueeze(1), lstm_out.unsqueeze(1))
        attn_output = attn_output.squeeze(1)

        cnn_in = x.squeeze(1).unsqueeze(1)
        cnn_out = self.conv1d(cnn_in)
        cnn_out = self.relu(cnn_out)
        cnn_out = torch.max_pool1d(cnn_out, kernel_size=cnn_out.size(2)).squeeze(2)

        combined_out = torch.cat((attn_output, cnn_out), dim=1)
        combined_out = self.dropout(combined_out)
        combined_out = self.relu(self.fc1(combined_out))
        output = self.fc2(combined_out)
        return output

# Training parameters
input_size = features_scaled.shape[1]
hidden_size = 32  # Increased hidden size
num_layers = 3  # Increased number of layers
cnn_channels = 32  # Increased CNN channels
attn_size = 64  # Increased attention size
dropout_rate = 0.3  # Dropout rate for regularization
batch_size = 64
learning_rate = 0.001
num_epochs = 10

# Dataset and DataLoader
dataset = CustomDataset(features_scaled, target)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Model, criterion, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = XLSTM_V2(input_size, hidden_size, num_layers, cnn_channels, attn_size, dropout_rate).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# # Training loop
start_time = time.time()

for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")


Epoch [1/10], Loss: 9.8661
Epoch [2/10], Loss: 5.0385
Epoch [3/10], Loss: 3.7772
Epoch [4/10], Loss: 5.7105
Epoch [5/10], Loss: 6.7234
Epoch [6/10], Loss: 0.7437
Epoch [7/10], Loss: 1.3912
Epoch [8/10], Loss: 1.3090
Epoch [9/10], Loss: 0.9219
Epoch [10/10], Loss: 1.2544
Training time: 729.24 seconds


In [8]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import time


features = train.drop(columns=['RUL', "unit", "Fc"]).values
target = train['RUL'].values.reshape(-1, 1)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

class CustomDataset(Dataset):
    def __init__(self, data, targets):
        self.data = torch.tensor(data, dtype=torch.float32).unsqueeze(1)  # Add sequence dimension if needed
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

class XLSTM_V2(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, cnn_channels, attn_size, dropout_rate=0.5):
        super(XLSTM_V2, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.layer_norm = nn.LayerNorm(hidden_size * 2)
        self.attention = nn.Linear(hidden_size * 2, attn_size)
        self.attn_combine = nn.Linear(attn_size, 1)
        self.conv1d = nn.Conv1d(in_channels=1, out_channels=cnn_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(hidden_size * 2 + cnn_channels, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out[:, -1, :]  # Use the output from the last time step
        lstm_out = self.layer_norm(lstm_out)

        attn_weights = torch.tanh(self.attention(lstm_out))
        attn_weights = self.attn_combine(attn_weights)
        attn_weights = torch.softmax(attn_weights, dim=1)
        attn_output = torch.bmm(attn_weights.unsqueeze(1), lstm_out.unsqueeze(1))
        attn_output = attn_output.squeeze(1)

        cnn_in = x.squeeze(1).unsqueeze(1)
        cnn_out = self.conv1d(cnn_in)
        cnn_out = self.relu(cnn_out)
        cnn_out = torch.max_pool1d(cnn_out, kernel_size=cnn_out.size(2)).squeeze(2)

        combined_out = torch.cat((attn_output, cnn_out), dim=1)
        combined_out = self.dropout(combined_out)
        combined_out = self.relu(self.fc1(combined_out))
        output = self.fc2(combined_out)
        return output

# Training parameters
input_size = features_scaled.shape[1]
hidden_size = 16  # Increased hidden size
num_layers = 3  # Increased number of layers
cnn_channels = 64  # Increased CNN channels
attn_size = 64  # Increased attention size
dropout_rate = 0.3  # Dropout rate for regularization
batch_size = 64
learning_rate = 0.001
num_epochs = 15

# Dataset and DataLoader
dataset = CustomDataset(features_scaled, target)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Model, criterion, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = XLSTM_V2(input_size, hidden_size, num_layers, cnn_channels, attn_size, dropout_rate).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
start_time = time.time()

for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")


Epoch [1/15], Loss: 17.3681
Epoch [2/15], Loss: 5.3047
Epoch [3/15], Loss: 4.3611
Epoch [4/15], Loss: 6.3605
Epoch [5/15], Loss: 7.9493
Epoch [6/15], Loss: 6.5318
Epoch [7/15], Loss: 3.7735
Epoch [8/15], Loss: 1.8394
Epoch [9/15], Loss: 1.3458
Epoch [10/15], Loss: 2.4808
Epoch [11/15], Loss: 1.9612
Epoch [12/15], Loss: 0.9142
Epoch [13/15], Loss: 2.1208
Epoch [14/15], Loss: 1.0382
Epoch [15/15], Loss: 1.7011
Training time: 1379.37 seconds


In [29]:
test

,unit,Fc,T24,T30,T48,T50,P15,P2,P21,P24,...,W31,W32,W48,W50,SmFan,SmLPC,SmHPC,phi,HPT_eff_mod,RUL
5263447,11.0,3.0,601.369822,1441.086963,1822.407728,1230.069061,15.900837,11.636834,16.142982,20.267629,...,26.649529,15.989717,217.085529,229.722454,16.745510,9.812495,25.345244,41.971419,-0.000698,58
5263448,11.0,3.0,601.381211,1441.055436,1822.376094,1230.025551,15.900690,11.637199,16.142833,20.267529,...,26.646358,15.987815,217.058720,229.694212,16.751997,9.806257,25.346932,41.971470,-0.000698,58
5263449,11.0,3.0,601.392126,1441.063188,1822.350721,1229.965758,15.899810,11.636655,16.141939,20.266716,...,26.644369,15.986621,217.043190,229.677650,16.758975,9.804009,25.348326,41.969940,-0.000698,58
5263450,11.0,3.0,601.348485,1440.964145,1822.141800,1229.809741,15.894349,11.632321,16.136395,20.260029,...,26.636854,15.982113,216.981145,229.612403,16.755378,9.803649,25.352080,41.964794,-0.000698,58
5263451,11.0,3.0,601.285695,1440.852510,1822.019760,1229.732630,15.886351,11.626363,16.128275,20.249683,...,26.625692,15.975415,216.890123,229.516137,16.753262,9.806697,25.351024,41.963540,-0.000698,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6517185,15.0,2.0,544.098139,1203.357895,1421.337905,1077.482738,12.837452,11.143805,13.032946,14.657219,...,15.786529,9.471918,127.892362,135.472311,16.461637,6.788187,32.223755,33.147993,-0.010829,0
6517186,15.0,2.0,544.035020,1203.472392,1422.772517,1078.697718,12.835965,11.144114,13.031436,14.652980,...,15.786545,9.471927,127.892061,135.471205,16.451330,6.803624,32.156314,33.208375,-0.010829,0
6517187,15.0,2.0,544.055787,1203.507712,1422.445115,1078.410015,12.838463,11.145616,13.033973,14.656059,...,15.791111,9.474666,127.930662,135.511892,16.444379,6.810375,32.172889,33.191835,-0.010829,0
6517188,15.0,2.0,544.060142,1203.529870,1422.576000,1078.539340,12.839843,11.146680,13.035374,14.657504,...,15.792761,9.475657,127.943031,135.524933,16.435526,6.810105,32.168495,33.197271,-0.010829,0


In [9]:
# Assuming df_unit1 is already defined
def evaluation(test):
    features = test.drop(columns=['RUL',"unit","Fc"]).values
    target = test['RUL'].values.reshape(-1, 1)
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    dataset = CustomDataset(features_scaled, target)
    test_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    with torch.no_grad():  # Disable gradient computation
        for inputs, targets in test_dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()

    # Calculate average test loss
    test_loss /= len(test_dataloader)
    return test_loss

In [10]:
test

,unit,Fc,T24,T30,T48,T50,P15,P2,P21,P24,...,W31,W32,W48,W50,SmFan,SmLPC,SmHPC,phi,HPT_eff_mod,RUL
5263447,11.0,3.0,601.369822,1441.086963,1822.407728,1230.069061,15.900837,11.636834,16.142982,20.267629,...,26.649529,15.989717,217.085529,229.722454,16.745510,9.812495,25.345244,41.971419,-0.000698,58
5263448,11.0,3.0,601.381211,1441.055436,1822.376094,1230.025551,15.900690,11.637199,16.142833,20.267529,...,26.646358,15.987815,217.058720,229.694212,16.751997,9.806257,25.346932,41.971470,-0.000698,58
5263449,11.0,3.0,601.392126,1441.063188,1822.350721,1229.965758,15.899810,11.636655,16.141939,20.266716,...,26.644369,15.986621,217.043190,229.677650,16.758975,9.804009,25.348326,41.969940,-0.000698,58
5263450,11.0,3.0,601.348485,1440.964145,1822.141800,1229.809741,15.894349,11.632321,16.136395,20.260029,...,26.636854,15.982113,216.981145,229.612403,16.755378,9.803649,25.352080,41.964794,-0.000698,58
5263451,11.0,3.0,601.285695,1440.852510,1822.019760,1229.732630,15.886351,11.626363,16.128275,20.249683,...,26.625692,15.975415,216.890123,229.516137,16.753262,9.806697,25.351024,41.963540,-0.000698,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6517185,15.0,2.0,544.098139,1203.357895,1421.337905,1077.482738,12.837452,11.143805,13.032946,14.657219,...,15.786529,9.471918,127.892362,135.472311,16.461637,6.788187,32.223755,33.147993,-0.010829,0
6517186,15.0,2.0,544.035020,1203.472392,1422.772517,1078.697718,12.835965,11.144114,13.031436,14.652980,...,15.786545,9.471927,127.892061,135.471205,16.451330,6.803624,32.156314,33.208375,-0.010829,0
6517187,15.0,2.0,544.055787,1203.507712,1422.445115,1078.410015,12.838463,11.145616,13.033973,14.656059,...,15.791111,9.474666,127.930662,135.511892,16.444379,6.810375,32.172889,33.191835,-0.010829,0
6517188,15.0,2.0,544.060142,1203.529870,1422.576000,1078.539340,12.839843,11.146680,13.035374,14.657504,...,15.792761,9.475657,127.943031,135.524933,16.435526,6.810105,32.168495,33.197271,-0.010829,0


In [18]:
evaluation(test)
# 79.8403990427653
# 80.28963890572237

72.0870618137673

In [11]:
import torch

# Assuming you have a model instance 'model'
# Save the model state_dict
torch.save(model.state_dict(), 'model_cnn_bi_better_rmse.pth')

# To load the model state_dict
# model.load_state_dict(torch.load('model_cnn_bi.pth'))
# model.eval()  

# residual connections, layer normalization, and deeper attention mechanisms.

In [91]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import time

# Assuming train is already defined and loaded with your training data
features = train.drop(columns=['RUL', "unit", "Fc"]).values
target = train['RUL'].values.reshape(-1, 1)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

class CustomDataset(Dataset):
    def __init__(self, data, targets):
        self.data = torch.tensor(data, dtype=torch.float32).unsqueeze(1)  # Add sequence dimension if needed
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

class XLSTM_V2(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, cnn_channels, attn_size, dropout_rate=0.5):
        super(XLSTM_V2, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.layer_norm = nn.LayerNorm(hidden_size * 2)
        self.attention = nn.Linear(hidden_size * 2, attn_size)
        self.attn_combine = nn.Linear(attn_size, 1)
        self.conv1d = nn.Conv1d(in_channels=1, out_channels=cnn_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(hidden_size * 2 + cnn_channels, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # LSTM
        lstm_out, _ = self.lstm(x)
        lstm_out = self.layer_norm(lstm_out)

        # Attention mechanism
        attn_weights = torch.tanh(self.attention(lstm_out))
        attn_weights = self.attn_combine(attn_weights).squeeze(-1)
        attn_weights = torch.softmax(attn_weights, dim=1)
        attn_weights = attn_weights.unsqueeze(1)
        attn_output = torch.bmm(attn_weights, lstm_out).squeeze(1)

        # CNN
        cnn_in = x.squeeze(1).unsqueeze(1)
        cnn_out = self.conv1d(cnn_in)
        cnn_out = self.relu(cnn_out)
        cnn_out = torch.max_pool1d(cnn_out, kernel_size=cnn_out.size(2)).squeeze(2)

        # Combine LSTM and CNN outputs
        combined_out = torch.cat((attn_output, cnn_out), dim=1)
        combined_out = self.dropout(combined_out)
        combined_out = self.relu(self.fc1(combined_out))
        output = self.fc2(combined_out)
        return output

# Training parameters
input_size = features_scaled.shape[1]
hidden_size = 32  # Increased hidden size
num_layers = 3  # Increased number of layers
cnn_channels = 32  # Increased CNN channels
attn_size = 64  # Increased attention size
dropout_rate = 0.3  # Dropout rate for regularization
batch_size = 64
learning_rate = 0.001
num_epochs = 10

# Dataset and DataLoader
dataset = CustomDataset(features_scaled, target)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Model, criterion, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = XLSTM_V2(input_size, hidden_size, num_layers, cnn_channels, attn_size, dropout_rate).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
start_time = time.time()

for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")


Epoch [1/10], Loss: 6.3233
Epoch [2/10], Loss: 2.5464
Epoch [3/10], Loss: 1.9659
Epoch [4/10], Loss: 1.8809
Epoch [5/10], Loss: 1.3529
Epoch [6/10], Loss: 0.8345
Epoch [7/10], Loss: 0.6030
Epoch [8/10], Loss: 0.3402
Epoch [9/10], Loss: 0.3885
Epoch [10/10], Loss: 0.5069
Training time: 719.12 seconds


#  added tranformer loss=84.9

In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import time

# Assuming train is already defined and loaded with your training data
features = train.drop(columns=['RUL', "unit", "Fc"]).values
target = train['RUL'].values.reshape(-1, 1)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

class CustomDataset(Dataset):
    def __init__(self, data, targets):
        self.data = torch.tensor(data, dtype=torch.float32).unsqueeze(1)  # Add sequence dimension if needed
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = nn.MultiheadAttention(embed_size, heads, dropout=dropout)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size)
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query):
        attention = self.attention(query, key, value)[0]
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

class XLSTM_V3(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, cnn_channels, attn_size, dropout_rate=0.5):
        super(XLSTM_V3, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.layer_norm1 = nn.LayerNorm(hidden_size * 2)
        self.transformer = TransformerBlock(embed_size=hidden_size * 2, heads=4, dropout=dropout_rate, forward_expansion=4)
        self.layer_norm2 = nn.LayerNorm(hidden_size * 2)
        self.conv1d = nn.Conv1d(in_channels=1, out_channels=cnn_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(hidden_size * 2 + cnn_channels, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # LSTM
        lstm_out, _ = self.lstm(x)
        lstm_out = self.layer_norm1(lstm_out)

        # Transformer Encoder
        transformer_out = self.transformer(lstm_out, lstm_out, lstm_out)
        transformer_out = self.layer_norm2(transformer_out)

        # CNN
        cnn_in = x.squeeze(1).unsqueeze(1)
        cnn_out = self.conv1d(cnn_in)
        cnn_out = self.relu(cnn_out)
        cnn_out = torch.max_pool1d(cnn_out, kernel_size=cnn_out.size(2)).squeeze(2)

        # Combine LSTM+Transformer and CNN outputs
        combined_out = torch.cat((transformer_out[:, -1, :], cnn_out), dim=1)
        combined_out = self.dropout(combined_out)
        combined_out = self.relu(self.fc1(combined_out))
        output = self.fc2(combined_out)
        return output

# Training parameters
input_size = features_scaled.shape[1]
hidden_size = 32  # Increased hidden size
num_layers = 3  # Increased number of layers
cnn_channels = 32  # Increased CNN channels
attn_size = 64  # Increased attention size
dropout_rate = 0.3  # Dropout rate for regularization
batch_size = 64
learning_rate = 0.001
num_epochs = 10

# Dataset and DataLoader
dataset = CustomDataset(features_scaled, target)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Model, criterion, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = XLSTM_V3(input_size, hidden_size, num_layers, cnn_channels, attn_size, dropout_rate).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
start_time = time.time()

for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")


Epoch [1/10], Loss: 3.3174
Epoch [2/10], Loss: 4.1060
Epoch [3/10], Loss: 1.9956
Epoch [4/10], Loss: 1.5094
Epoch [5/10], Loss: 2.8562
Epoch [6/10], Loss: 1.2577
Epoch [7/10], Loss: 0.8690
Epoch [8/10], Loss: 1.8958
Epoch [9/10], Loss: 0.4142
Epoch [10/10], Loss: 0.7861
Training time: 982.49 seconds


In [11]:
evaluation(test)

64.95948318765747

In [72]:
len(test)

1253743

In [79]:
train.columns

Index(['unit', 'Fc', 'T24', 'T30', 'T48', 'T50', 'P15', 'P2', 'P21', 'P24',
       'Ps30', 'P40', 'P50', 'Nf', 'Nc', 'Wf', 'T40', 'P30', 'P45', 'W21',
       'W22', 'W25', 'W31', 'W32', 'W48', 'W50', 'SmFan', 'SmLPC', 'SmHPC',
       'phi', 'LPC_flow_mod', 'HPT_eff_mod', 'RUL'],
      dtype='object')

In [30]:
test

,unit,Fc,T24,T30,T48,T50,P15,P2,P21,P24,...,W32,W48,W50,SmFan,SmLPC,SmHPC,phi,LPC_flow_mod,HPT_eff_mod,RUL
5263447,11.0,3.0,601.369822,1441.086963,1822.407728,1230.069061,15.900837,11.636834,16.142982,20.267629,...,15.989717,217.085529,229.722454,16.745510,9.812495,25.345244,41.971419,0.0,-0.000698,58
5263448,11.0,3.0,601.381211,1441.055436,1822.376094,1230.025551,15.900690,11.637199,16.142833,20.267529,...,15.987815,217.058720,229.694212,16.751997,9.806257,25.346932,41.971470,0.0,-0.000698,58
5263449,11.0,3.0,601.392126,1441.063188,1822.350721,1229.965758,15.899810,11.636655,16.141939,20.266716,...,15.986621,217.043190,229.677650,16.758975,9.804009,25.348326,41.969940,0.0,-0.000698,58
5263450,11.0,3.0,601.348485,1440.964145,1822.141800,1229.809741,15.894349,11.632321,16.136395,20.260029,...,15.982113,216.981145,229.612403,16.755378,9.803649,25.352080,41.964794,0.0,-0.000698,58
5263451,11.0,3.0,601.285695,1440.852510,1822.019760,1229.732630,15.886351,11.626363,16.128275,20.249683,...,15.975415,216.890123,229.516137,16.753262,9.806697,25.351024,41.963540,0.0,-0.000698,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6517185,15.0,2.0,544.098139,1203.357895,1421.337905,1077.482738,12.837452,11.143805,13.032946,14.657219,...,9.471918,127.892362,135.472311,16.461637,6.788187,32.223755,33.147993,0.0,-0.010829,0
6517186,15.0,2.0,544.035020,1203.472392,1422.772517,1078.697718,12.835965,11.144114,13.031436,14.652980,...,9.471927,127.892061,135.471205,16.451330,6.803624,32.156314,33.208375,0.0,-0.010829,0
6517187,15.0,2.0,544.055787,1203.507712,1422.445115,1078.410015,12.838463,11.145616,13.033973,14.656059,...,9.474666,127.930662,135.511892,16.444379,6.810375,32.172889,33.191835,0.0,-0.010829,0
6517188,15.0,2.0,544.060142,1203.529870,1422.576000,1078.539340,12.839843,11.146680,13.035374,14.657504,...,9.475657,127.943031,135.524933,16.435526,6.810105,32.168495,33.197271,0.0,-0.010829,0


# testing instance by instance to make sure

In [32]:
# Select a specific row, e.g., row 10000
row_index = 1253742
row_data = test.iloc[row_index]

# Extract features for the selected row
row_features = row_data.drop(['RUL', 'unit', 'Fc']).values.reshape(1, -1)

# Scale the features
row_features_scaled = scaler.transform(row_features)

# Convert to tensor
row_features_tensor = torch.tensor(row_features_scaled, dtype=torch.float32).unsqueeze(0).to(device)  # Add batch dimension

# Make predictions
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient computation
    prediction = model(row_features_tensor)

# Move the prediction back to CPU and convert to numpy
prediction = prediction.cpu().numpy()

print(f'Predicted RUL for row {row_index}: {prediction[0][0]:.4f}')


Predicted RUL for row 1253742: 3.0048


In [44]:
predictions=[]
truth=[]
for i in range(len(test)):
    row_index = i
    row_data = test.iloc[row_index]

# Extract features for the selected row
    row_features = row_data.drop(['RUL', 'unit', 'Fc']).values.reshape(1, -1)

# Scale the features
    row_features_scaled = scaler.transform(row_features)

# Convert to tensor
    row_features_tensor = torch.tensor(row_features_scaled, dtype=torch.float32).unsqueeze(0).to(device)  # Add batch dimension

# Make predictions
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient computation
        prediction = model(row_features_tensor)

# Move the prediction back to CPU and convert to numpy
    prediction = prediction.cpu().numpy()
    predictions.append(prediction)
    truth.append(row_data["RUL"])
    

    # print(f'Predicted RUL for row {row_index}: {prediction[0][0]:.4f}')
    
    

In [73]:
def calculate_mse(true_values, estimated_values):
    # Check if the arrays have the same length
    if len(true_values) != len(estimated_values):
        raise ValueError("Arrays must have the same length")

    # Calculate the squared differences
    squared_diff = [(true_val - est_val) ** 2 for true_val, est_val in zip(true_values, estimated_values)]

    # Calculate the mean of squared differences
    mse = sum(squared_diff) / len(true_values)

    return mse

# Example usage:
true_values = [3, 5, 7, 9]
estimated_values = [2, 4, 6, 8]

mse = calculate_mse(truth,temp)
print("Mean Squared Error:", mse)


Mean Squared Error: 38.92970765621519


In [7]:
import torch

# Assuming you have a model instance 'model'
# Save the model state_dict
# torch.save(model.state_dict(), 'model.pth')

# To load the model state_dict
model.load_state_dict(torch.load('model_cnn_bi.pth'))
model.eval()  

NameError: name 'model' is not defined

In [8]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import time

# Assuming df_unit1 is already defined
features = train.drop(columns=['RUL', "unit", "Fc"]).values
target = train['RUL'].values.reshape(-1, 1)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

class CustomDataset(Dataset):
    def __init__(self, data, targets):
        self.data = torch.tensor(data, dtype=torch.float32).unsqueeze(1)  # Add sequence dimension if needed
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

class XLSTM_V2(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, cnn_channels, attn_size, dropout_rate=0.5):
        super(XLSTM_V2, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.layer_norm = nn.LayerNorm(hidden_size * 2)
        self.attention = nn.Linear(hidden_size * 2, attn_size)
        self.attn_combine = nn.Linear(attn_size, 1)
        self.conv1d = nn.Conv1d(in_channels=1, out_channels=cnn_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(hidden_size * 2 + cnn_channels, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out[:, -1, :]  # Use the output from the last time step
        lstm_out = self.layer_norm(lstm_out)

        attn_weights = torch.tanh(self.attention(lstm_out))
        attn_weights = self.attn_combine(attn_weights)
        attn_weights = torch.softmax(attn_weights, dim=1)
        attn_output = torch.bmm(attn_weights.unsqueeze(1), lstm_out.unsqueeze(1))
        attn_output = attn_output.squeeze(1)

        cnn_in = x.squeeze(1).unsqueeze(1)
        cnn_out = self.conv1d(cnn_in)
        cnn_out = self.relu(cnn_out)
        cnn_out = torch.max_pool1d(cnn_out, kernel_size=cnn_out.size(2)).squeeze(2)

        combined_out = torch.cat((attn_output, cnn_out), dim=1)
        combined_out = self.dropout(combined_out)
        combined_out = self.relu(self.fc1(combined_out))
        output = self.fc2(combined_out)
        return output

# Training parameters
input_size = features_scaled.shape[1]
hidden_size = 32  # Increased hidden size
num_layers = 3  # Increased number of layers
cnn_channels = 32  # Increased CNN channels
attn_size = 64  # Increased attention size
dropout_rate = 0.3  # Dropout rate for regularization
batch_size = 64
learning_rate = 0.001
num_epochs = 10

# Dataset and DataLoader
dataset = CustomDataset(features_scaled, target)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Model, criterion, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = XLSTM_V2(input_size, hidden_size, num_layers, cnn_channels, attn_size, dropout_rate).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [9]:
import torch

# Assuming you have a model instance 'model'
# Save the model state_dict
# torch.save(model.state_dict(), 'model.pth')

# To load the model state_dict
model.load_state_dict(torch.load('model_cnn_bi.pth'))
model.eval()  

XLSTM_V2(
  (lstm): LSTM(30, 32, num_layers=3, batch_first=True, bidirectional=True)
  (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (attention): Linear(in_features=64, out_features=64, bias=True)
  (attn_combine): Linear(in_features=64, out_features=1, bias=True)
  (conv1d): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (relu): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=96, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=1, bias=True)
)

In [11]:
from node2vec import Node2Vec
import networkx as nx

# Create a graph
G = nx.fast_gnp_random_graph(n=100, p=0.5)

# Initialize Node2Vec
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=4)

# Fit model
model = node2vec.fit(window=10, min_count=1, batch_words=4)

# Get embeddings
embeddings = {str(node): model.wv[str(node)] for node in G.nodes()}

# Print the embedding for node 0
print(embeddings['0'])


ImportError: cannot import name 'triu' from 'scipy.linalg' (c:\Users\lenovo legion\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\linalg\__init__.py)

In [6]:
# %pip install --upgrade pip
# %pip install --upgrade node2vec networkx gensim scipy
%pip install pytorch_geometric


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pytorch_geometric (from versions: none)
ERROR: No matching distribution found for pytorch_geometric


In [13]:
import torch
import torch_geometric
from torch_geometric.data import Data

# Normalize the dataframe if necessary
df_normalized = (train - train.mean()) / train.std()

# Convert dataframe to tensor
data_tensor = torch.tensor(df_normalized.values, dtype=torch.float)

# Number of time steps and features
num_time_steps, num_features = data_tensor.shape

# Create nodes: each (time step, feature) pair is a node
# We can create an edge list based on temporal adjacency (each time step is connected to the next)
edge_index = []

# Connect each time step to the next for each feature
for feature in range(num_features):
    for time_step in range(num_time_steps - 1):
        start = time_step * num_features + feature
        end = (time_step + 1) * num_features + feature
        edge_index.append([start, end])

# Convert edge_index to tensor
edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

# Feature matrix for the graph: each node's features are the corresponding values in the dataframe
x = data_tensor.reshape(-1, 1)  # each node gets its corresponding value as feature

# Assume RUL is provided in a separate array
# Replace 'your_rul_values' with your actual RUL values, here we assume it is a list or an array
your_rul_values = train["RUL"]  # replace with your actual RUL values
rul = torch.tensor(your_rul_values, dtype=torch.float)

# Create a Data object
data = Data(x=x, edge_index=edge_index, y=rul)


In [15]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Initialize the model, loss function, and optimizer
model = GCN(in_channels=1, hidden_channels=16, out_channels=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

# Training loop
model.train()
for epoch in range(10):  # Number of epochs
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out, data.y)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


c:\Users\lenovo legion\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([853142])) that is different to the input size (torch.Size([28153686, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 96076367925648 bytes.